# Week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
load_dotenv(override=True)

In [ ]:

OLLAMA_BASE_URL = "http://localhost:11434/v1"
OPEN_ROUTER_BASE_URL = "https://openrouter.ai/api/v1"
OPEN_ROUTER_API_KEY = os.getenv('OPEN_ROUTER_API_KEY')

cs_client = OpenAI(
    base_url=OPEN_ROUTER_BASE_URL,
    api_key=OPEN_ROUTER_API_KEY
)

os_client = OpenAI(
    base_url=OLLAMA_BASE_URL,
    api_key='ollama'
)

CS_MODEL = 'gpt-5-nano'
OS_MODEL = 'llama3.2'

SYSTEM_PROMPT = (
    "You are a Senior Technical Instructor. "
    "Explain complex concepts simply, use analogies, and provide "
    "a short code snippet or mathematical proof if applicable."
)

In [ ]:
def explain_with_closed_source_model(question: str, model=CS_MODEL):
    """Fetches explanation from Closed Source Models (openai/gpt-4.1-mini, anthropic/claude-sonnet-4.6, google/gemini-3-flash-preview etc)."""
    try:
        print(f"--- 🧠 Technical Question: {question} ---\n")

        print(f"🚀 Fetching {model} Explanation...")
        stream = cs_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": question}
            ],
            stream=True
        )
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            update_display(Markdown(response), display_id=display_handle.display_id)
        return response
    except Exception as e:
        return f"Closed Source Model Error: {e}"

In [ ]:
# Test it out with closed source model

model = 'anthropic/claude-sonnet-4.6'
question = "What is the difference between a Vector Database and a Relational Database in the context of RAG?"
explain_with_closed_source_model(question, model)

In [ ]:
def explain_with_open_source_model(question: str, model=OS_MODEL):
    """Fetches explanation from Open Source Models (llama-3.2, gemma3, mistral etc.)."""
    try:
        print(f"--- 🧠 Technical Question: {question} ---\n")

        print(f"🚀 Fetching {model} Explanation...")
        response = os_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": question}
            ],
        )
        result = response.choices[0].message.content
        return display(Markdown(result))
    except Exception as e:
        return f"Open Source Model Error: {e}"

In [ ]:
# Test it out with open source model

model = 'gemma3:270m'
question = "What is the difference between a Vector Database and a Relational Database in the context of RAG?"
explain_with_open_source_model(question, model)